<h1><center> Tutorial 1: Configuring Algorithms for Testing </center></h1>

---

This tutorial teaches you how to implement optimization algorithms for benchmarking with pyGOLD. You'll learn the `BaseOptimizer` interface and see examples implementing multiple types of algorithms.

In [1]:
import numpy as np
from pygold.optimizer import BaseOptimizer, OptimizationResult
import pygold

## Part 1: The BaseOptimizer Interface
---

All algorithms in pyGOLD must inherit from `BaseOptimizer` and implement:

1. `deterministic` (bool): Whether the algorithm produces the same result given the same inputs
2. `n_points` (int): How many initial points the algorithm needs
3. `optimize()` method: The main optimization function

The attribute `n_points` defines how many initial points the benchmark runner will provide to the algorithm.

- `n_points = 0`: Algorithm doesn't need initial points (e.g., a deterministic region exploration algorithm)
- `n_points = 1`: Algorithm needs a single starting point (e.g., a basin-hopping algorithm)
- `n_points > 1`: Algorithm needs multiple initial points (e.g., a population-based algorithm)

## Part 2: SciPy Examples
---

Let's start with wrapping some scipy algorithms. In addition to the `deterministic` and `n_points` attributes, each algorithm implements the optimize method with the signature: 

> optimize(self, func, bounds, x0=None, constraints=None, **kwargs)

Your algorithm may not require an initial point or support constraints. The optimize method is still expected to accept them as arguments. 

In [2]:
from scipy.optimize import differential_evolution, basinhopping, dual_annealing, shgo

# Dual Annealing: stochastic, no initial conditions
class DualAnnealing(BaseOptimizer):
    deterministic = False
    n_points = 0

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        result = dual_annealing(func, bounds, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

# Basin Hopping: stochastic, single initial condition
class BasinHopping(BaseOptimizer):
    deterministic = False
    n_points = 1

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        minimizer_kwargs = {'bounds': bounds}
        result = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

# Differential Evolution: stochastic, multiple initial conditions
class DifferentialEvolution(BaseOptimizer):
    deterministic = False
    n_points = 15

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        result = differential_evolution(func, bounds, init=x0, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

# SHGO: deterministic, no initial conditions
class SHGO(BaseOptimizer):
    deterministic = True
    n_points = 0

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        # Configure parameters
        shgo_kwargs = {
            'n': 100,
            'iters': 5,
            'sampling_method': 'sobol',
            'minimizer_kwargs': {'method': 'L-BFGS-B', 'options': {'ftol': 1e-8}}
        }

        result = shgo(func, bounds, **shgo_kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)


## Part 3: A More Complex Example, The Pymoo Genetic Algorithm
---

Now let's implement a more sophisticated algorithm wrapper using the pymoo library. This is more complicated than the previous examples because the pymoo library requires problems to be defined as subclasses of the Problem class within the pymoo library. We must therefore create the problem and configure the algorithm within the optimize method. Apart from this, the approach is identical to the previous examples.

In [3]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize

class GeneticAlgorithm(BaseOptimizer):
    deterministic = False  # GA is stochastic
    n_points = 0           # Generates its own population

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):

        # pymoo requires a problem defined as a subclass of Problem
        class PymooProblem(ElementwiseProblem):
            def __init__(self, objective_func, bounds):
                n_var = len(bounds)
                xl = np.array([b[0] for b in bounds])
                xu = np.array([b[1] for b in bounds])
                super().__init__(n_var=n_var, n_obj=1, xl=xl, xu=xu)
                self.objective_func = objective_func
                self.n_calls = 0

            def _evaluate(self, x, out, *args, **kwargs):
                self.n_calls += 1
                out["F"] = self.objective_func(x)

        # Create the problem
        problem = PymooProblem(func, bounds)

        # Configure the algorithm
        algorithm = GA(pop_size=100, eliminate_duplicates=True)

        # Run optimization
        res = minimize(problem, algorithm, ('n_gen', 100), verbose=False)

        return OptimizationResult(x=res.X, fun=res.F[0], algorithm=self.name)

## Part 4: Validate Implementations
---

Let's validate our algorithm implementations by running them on a test problem and displaying the number of function evaluations used.

In [4]:
# Get a simple test problem
problems = pygold.get_standard_problems(["2D", "Unconstrained"])
test_problem = problems[3]  # Just use one

print(f"Testing with problem: {test_problem.__name__}")

# Create instances of our algorithms
solvers = [
    DualAnnealing(),
    BasinHopping(),
    DifferentialEvolution(),
    SHGO(),
    GeneticAlgorithm()
]

# Quick test run
pygold.run_solvers(solvers=solvers, problems=[test_problem], test_dimensions=[2], n_iters=2, output_folder="algorithm_test", verbose=True)

# Postprocess performance data
res = pygold.postprocess_data(["algorithm_test/BasinHopping", "algorithm_test/DifferentialEvolution", "algorithm_test/DualAnnealing", "algorithm_test/GeneticAlgorithm", "algorithm_test/SHGO"])

# Display results
display(res['data']['mean_fevals'])

[codecarbon WARNING @ 11:48:25] Multiple instances of codecarbon are allowed to run at the same time.


Testing with problem: Schaffer4
Running DualAnnealing on Schaffer4 in 2D, iteration 1/2
Running DualAnnealing on Schaffer4 in 2D, iteration 2/2
Running BasinHopping on Schaffer4 in 2D, iteration 1/2
Running BasinHopping on Schaffer4 in 2D, iteration 2/2
Running DifferentialEvolution on Schaffer4 in 2D, iteration 1/2
Running DifferentialEvolution on Schaffer4 in 2D, iteration 2/2
Running SHGO on Schaffer4 in 2D, iteration 1/1
Running GeneticAlgorithm on Schaffer4 in 2D, iteration 1/2
Running GeneticAlgorithm on Schaffer4 in 2D, iteration 2/2


,problem,solver,n_dims,target_0.1,target_0.01,target_0.0001,target_1e-08
0,Schaffer4,BasinHopping,2,1822.0,3316.0,3484.0,3910.0
1,Schaffer4,DifferentialEvolution,2,53.5,229.5,913.0,NaN
2,Schaffer4,DualAnnealing,2,186.5,897.5,1118.5,2013.0
3,Schaffer4,GeneticAlgorithm,2,287.5,997.5,3278.0,6302.0
4,Schaffer4,SHGO,2,662.0,6675.0,NaN,NaN


## Part 5: Additional Notes
---
### Handle Constraints

If your solver supports constraints and you indend to test it with constrained problems, ensure you handle the constraints properly. The constraint functions will return negative values when violated. 

For example, to run SHGO with constraints:

In [5]:
class ShgoConstraints(BaseOptimizer):
    deterministic = True
    n_points = 0

    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        # Convert constraints to the format expected by shgo
        cons = [{'type': 'ineq', 'fun': c} for c in constraints] if constraints else []

        result = shgo(func, bounds, constraints=cons, **kwargs)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)

### Common Implementation Pattern

Most algorithms will have an implementation similar to this:

```python
class MyAlgorithm(BaseOptimizer):
    deterministic = (bool)
    n_points = (int)
    
    def optimize(self, func, bounds, x0=None, constraints=None, **kwargs):
        # Call your algorithm
        result = my_algorithm(func, bounds, x0)
        return OptimizationResult(result.x, result.fun, algorithm=self.name)
```

## Summary
---

You now know how to:
- Implement algorithms from the `BaseOptimizer` interface
- Handle different initial condition requirements
- Distinguish deterministic vs stochastic algorithms
- Handle constraints properly

Next: **Tutorial 2** will show you how to run complete experiments with your algorithms.